### loading modules and data

In [1]:
from models import *
from utils import *
from data_utils import *

dataset_path = './data/'
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)



GPU is available
Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold Training and Validation

In [2]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
vote_num = 20

best_val_acc = 0.0
for k in range(1):
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='CNNLSTM', dropout=0.3)
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=10, aug_type=aug_type, window_size=window_size, vote_num=vote_num)
print ('average best validation accuracy of 5 folds is :{}'.format(best_val_acc))

fold 1
epoch: 1      time: 17.13    loss: 729.175    train acc: 0.281    val acc: 0.307
saving best model...
epoch: 2      time: 17.96    loss: 719.919    train acc: 0.311    val acc: 0.317
saving best model...
epoch: 3      time: 17.86    loss: 707.840    train acc: 0.344    val acc: 0.404
saving best model...
epoch: 4      time: 17.74    loss: 700.896    train acc: 0.355    val acc: 0.366
epoch: 5      time: 17.60    loss: 683.591    train acc: 0.382    val acc: 0.383


KeyboardInterrupt: 

### Training and Testing

In [4]:
X_train, y_train, p_train = X_train_valid, y_train_valid, person_train_valid
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
TrainValRNN(model, criterion, optimizer, EEG_trainloader, num_epochs=10, aug_type=aug_type, window_size=window_size, vote_num=vote_num

TestRNN(model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4221
